Итак, ОСС хорошо подходят для определения эмоциональной окраски текста, но плохо - для предсказаний температуры.

В данном ноутбуке представлено универсальное решение.

ОСС обрабатывает ооочень длинну. последовательность, сжимая ее размерность, а затем полученные данные помещаются в RNN, которая работает с большей точностью

Предсказание температуры с помощью объединения ОСС и RNN

In [ ]:
# все данные идентичны ноутбуку RNN_jena

import pandas as pd
import numpy as np

data_path = '/content/drive/My Drive/StudyingKeras/RNN/jena_climate_data_2009_20016/jena_climate_2009_2016.csv'

# Этот код выведет 420 551 строку с данными - каждая строка соответствует
# одному замеру и содержит дату замера и 14 значений разных параметров
data = pd.read_csv(data_path)


with open(data_path, 'r') as f:
  data = f.read()

# получение всех строк 
lines = data.split('\n')
# получение заголовков всех столбцов
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
  # каждое значение в строке отделено запятой
  values = [float(x) for x in line.split(',')[1:]]
  # все переведенные в float числа помещает в массив
  float_data[i, :] = values

# все данные измеряются в разных диапозонах (температура, давление...)
# поэтому нормализуем их
mean = float_data[:200_000].mean(axis=0)
float_data -= mean
std = float_data[:200_000].std(axis=0)
float_data /= std

# принимает: текущий массив данных
# возвращает: пакеты, представляющие собой недавнее прошлое, предсказанную температуру
# data - нормализованные данные
# lookback - количество интервалов из прошлого
# delay - количество интервалов из будущего
# min_index, max_index - границы в data, в которых выбираются значения
# shuffle - извлекать по порядку или с перемешиванием
# batch_size - количество образцов в пакете
# step - период в интервалах, в котором выбираются значения (6 - значит каждый час)

def generator(data, lookback, delay, min_index, max_index, 
              shuffle=False, batch_size=128, step=6):
  # если правая граница не указана - она определяется сама
  if max_index is None:
    max_index = len(data) - delay - 1
  # правая граница первого интервала
  i = min_index + lookback
  while True:
    if shuffle:
      # если данные не извлекаются по порядку, то генерируем список случайных
      # строк, откуда эти данные будем брать
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size > max_index:
        # если дошли до конца (до самой правой границы), то возвращаемся в начало
        i = min_index + lookback
      # генерируем список подряд идущиъ индексов строк
      rows = np.arange(i, min(i + batch_size, max_index))
      # смещаем правую границу первого интервала к концу только что созданного списка
      # так как мы его будем обрабатывать и больше его начало нас не интересует
      i += len(rows)
    # создаем тензоры образцов и целей
    # все это - просто часть известных данных из файла
    # samples - данные из прошлого
    # targets - данные (как бы) из будущего (просто правее)
    samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      # берем lookback значений из прошлого с интервалом step
      indices = range(rows[j] - lookback, rows[j], step)
      # извлекаем образцы из прошлого по индексам
      samples[j] = data[indices]
      # извлекаем цели (которые тоже образцы) из будущего
      targets[j] = data[rows[j] + delay][1]
    # образцы возвращаются в прямом порядке
    # для большинства временных последовательностей если сеть обрабатывает последовательность в прямом порядке
    # то она лучше обучается, чем если в обратном
    yield samples, targets    

# 100 дней назад
lookback = 14400
# замеры раз в полчаса
step = 3
# вперед на день
delay = 144
batch_size = 128

train_generator = generator(
    float_data, 
    lookback=lookback,
    delay=delay,
    min_index=0,
    max_index=200_000,
    shuffle=True,
    step=step,
    batch_size=batch_size
)

val_generator = generator(
    float_data, 
    lookback=lookback,
    delay=delay,
    min_index=200_001,
    max_index=300_000,
    shuffle=True,
    step=step,
    batch_size=batch_size
)

test_generator = generator(
    float_data, 
    lookback=lookback,
    delay=delay,
    min_index=300_000,
    max_index=None,
    shuffle=True,
    step=step,
    batch_size=batch_size
)

# сколько раз надо обратиться к val_generator, чтобы получить проверочный набор целиком
val_steps = (300_000 - 200_001 - lookback) // batch_size
# сколько раз надо обратиться к test_generator, чтобы получить тестовый набор целиком
test_steps = (len(float_data) - 300_001 - lookback) // batch_size

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GRU

model = Sequential()
model.add(Conv1D(32, 5, activation='relu', input_shape=(None, float_data.shape[-1])))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 5, activation='relu'))
model.add(GRU(32, dropout=0.1))
model.add(Dense(1))
model.summary()

model.compile(optimizer='rmsprop', loss='mae')

history = model.fit(
    train_generator,
    steps_per_epoch=500,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_steps
).history

In [ ]:
def draw_graph(history):
  import matplotlib.pyplot as plt
  loss = history['loss']
  val_loss = history['val_loss']
  epochs = range(1, len(loss) + 1)
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

draw_graph(history)

Такая архитектура не дотягивает до решения с регуляризированным GRU, однако он действует намного быстрее